## A PySpark wordcount example

This example aims to show in some more detail how MapReduce is used on Resiliant Distributed Datasets (RDD) in Spark to count words. You can add `.collect()` to the result of each transformation to see the intermediate output.

See the Python [documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html) on RDDs for more information. Note, that if your data has structure - something like .csv or regular JSON you should probably use [DataFrames](https://spark.apache.org/docs/latest/sql-programming-guide.html).

### Import Spark libraries
Import the necessary Spark libraries. The entry point is always the `SparkSession` instance. If you run the `pyspark` shell then this session instance will already have been created for you. It's stored in the `spark` variable.

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession \
    .builder \
    .appName("Python Spark wordcount") \
    .getOrCreate()
    
spark.conf.set("spark.sql.shuffle.partitions", "5")

We read in text. This creates an RDD.

In [14]:
lines = spark.sparkContext.parallelize(['I love Hadoop,','but I do not love Hadopi.'])

The first transformation replaces non-letters with spaces and lower cases text.

In [15]:
lines_clean = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower())

The second transformation splits each line into words and then merges all words together.

Input:

```
[
    "i love Hadoop"
    "but i do not love Hadopi"
]
```

Output:

```
["i", "love", "hadoop", "but", "i", "do", "not", "love", "hadopi"]
```

If we had used `map` instead of `flatMap` the output would have been:

```
[
    ["i", "love", "hadoop"]
    ["but", "i", "do", "not", "love", "hadopi"]
]
```



In [16]:
words_flat = lines_clean.flatMap(lambda x: x.split())

The third transformation emits a key-value pair `(word, 1)` for each word in the flat list. In Spark a key-value pair must be of type tuple.

In [17]:
def mapWord(x):
    return (x,1)

words_mapped = words_flat.map(lambda x: (x, 1))
words_mapped.collect()

[('i', 1),
 ('love', 1),
 ('hadoop', 1),
 ('but', 1),
 ('i', 1),
 ('do', 1),
 ('not', 1),
 ('love', 1),
 ('hadopi', 1)]

The fourth transformation groups all key-value pairs by key and sums the values.

In [18]:
temporary_result = words_mapped.reduceByKey(lambda x,y:x+y)
temporary_result.collect()

[('i', 2),
 ('not', 1),
 ('love', 2),
 ('do', 1),
 ('hadoop', 1),
 ('hadopi', 1),
 ('but', 1)]

The fifth transformations simply revert the `(word, sum(word))` key-value pair so that the word becomes the value and the sum the key.

In [19]:
reversed_result = temporary_result.map(lambda x:(x[1],x[0]))

The final transformation sorts the keys (sum) in descending order.

In [20]:
sorted_results = reversed_result.sortByKey(False)

We finally claim the result by calling an action - retrieve the top 3 elements.

In [21]:
sorted_results.take(10)

[(2, 'i'),
 (2, 'love'),
 (1, 'not'),
 (1, 'do'),
 (1, 'hadoop'),
 (1, 'hadopi'),
 (1, 'but')]

Now do everything in one operation:

In [22]:
lines \
    .map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False) \
    .take(10)

[(2, 'i'),
 (2, 'love'),
 (1, 'not'),
 (1, 'do'),
 (1, 'hadoop'),
 (1, 'hadopi'),
 (1, 'but')]